List assets file and find those that have a minerva story but do not have a thumbnails

In [ ]:
import json
import pandas as pd
import imageio

In [ ]:
assets = pd.read_json('assets/htan-imaging-assets.json')


In [ ]:
assets

Find the 'top' of each group in a minerva story and render into a gif



In [ ]:
example_path = '/Users/ataylor/Documents/projects/htan/htan-artist/work/55/60af1f3d89e0bf4bdd2999e38ff925/minerva'

In [ ]:
from pathlib import Path
import os
import re
import glob

my_dict = {'path':[],'group':[],'level':[]}

for path in glob.iglob(example_path + '**/**/*_0_0.jpg', recursive=True):
    group = re.search(r'Group-(\d+)',path).group(1)
    my_dict['group'].append(int(group))
#
    level = re.search(r'(\d+)_0_0.jpg',path).group(1)
    my_dict['level'].append(int(level)) 

    my_dict['path'].append(path)    

df = pd.DataFrame(my_dict)
df.groupby('group')['level'].max()

In [ ]:
top_images = []

for group in glob.iglob(example_path + '**/Group*', recursive=True):
    levels = []
    images = list(glob.iglob(group + '**/*_0_0.jpg', recursive=True))
    for image in images:
        level = re.search(r'(\d+)_0_0.jpg',image).group(1)
        levels.append(level)
    max_level = max(levels)
    max_level_idx = levels.index(max_level)
    max_image = images[max_level_idx]
    top_images.append(max_image)

sorted(top_images)

imageio.mimsave('test.gif', list(map(imageio.imread, top_images)), duration = 0.5)

    



In [ ]:
list(glob.iglob(example_path + '**/**/*_0_0.jpg', recursive=True))

In [ ]:
import imageio


In [ ]:
import boto3
import re

In [ ]:
session = boto3.session.Session(profile_name = 'htan-dev')
s3 = session.resource('s3')
bucket = s3.Bucket('htan-assets')

In [ ]:
def get_minerva_thumbnail(minerva_url):


    cloudflare = 'https://d3p249wtgzkn5u.cloudfront.net/'
    prefix = minerva_url.replace(cloudflare,'').replace('/minerva/index.html','')

    thumbs = [] 

    for obj in bucket.objects.filter(Prefix = prefix):
        if obj.key.endswith('_0_0.jpg'):
            thumbs.append(obj.key)

    if re.search('loving_swartz_2',thumbs[0]):
        selected = thumbs[2]
    elif re.search('naughty_reubens',thumbs[0]):
        selected = thumbs[2]
    else:
        selected = sorted(thumbs)[-1]

    thumbnail_url = cloudflare + selected

    return(thumbnail_url)



get_minerva_thumbnail('https://d3p249wtgzkn5u.cloudfront.net/synid/syn25547797/loving_swartz_2/minerva/index.html')

In [ ]:
re.match(
    'synid/syn25547797/loving_swartz_2/minerva/Group-10_36__TargetMYC--37__TargetNa--38__TargetP--39__Targetp53/0_0_0.jpg', 
    r'.+loving_swartz_2.+'
    )

In [ ]:
assets = pd.read_json('assets/htan-imaging-assets.json')

new_assets = assets #assets[assets['thumbnail'].isnull()]

new_assets['thumbnail'] = new_assets.apply(lambda x: get_minerva_thumbnail(x['minerva']) if(pd.isnull(x['thumbnail'])) else x['thumbnail'], axis = 1)

new_assets

In [ ]:
new_assets.to_csv('minerva-thumbs.csv', index=False)

In [ ]:
new_assets.to_json('test.json', orient='records', indent = 4, lines = True)

In [ ]:
with open("test.json", "w+") as f:
    json.dump(new_assets.to_dict(orient="record"), f, indent=4)